In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from openinference.instrumentation.openai import OpenAIInstrumentor

from phoenix.otel import register

In [ ]:
tracer_provider = register(endpoint="http://127.0.0.1:6006/v1/traces")
OpenAIInstrumentor(tracer_provider=tracer_provider).instrument()

In [ ]:
data = SimpleDirectoryReader(input_dir="/Users/xandersong/Desktop/data/paul_graham/").load_data()
index = VectorStoreIndex.from_documents(data)

from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about an essay discussing Paul Grahams life."
    ),
)

"""Chat with your data"""

response = chat_engine.chat("Hello!")

print(response)

"""Ask a follow up question"""

response = chat_engine.chat("What did Paul Graham do growing up?")

print(response)

response = chat_engine.chat("Can you tell me more?")

print(response)

"""Reset conversation state"""

chat_engine.reset()

response = chat_engine.chat("Hello! What do you know?")

print(response)

"""## Streaming Support"""

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI as LlamaIndexOpenAI

llm = LlamaIndexOpenAI(model="gpt-3.5-turbo", temperature=0)
data = SimpleDirectoryReader(input_dir="./data/paul_graham/").load_data()

index = VectorStoreIndex.from_documents(data)

chat_engine = index.as_chat_engine(chat_mode="context", llm=llm)

response = chat_engine.stream_chat("What did Paul Graham do after YC?")
for token in response.response_gen:
    print(token, end="")